# Проект "Flat price prediction": Извлечение датасета

In [1]:
import os
import numpy as np
import pandas as pd
import sys
import re
import time
from bs4 import BeautifulSoup    
import requests 

In [2]:
#Используем простой звуковой индикатор для облегчения контроля за процессом
import jupyter_beeper
beeper = jupyter_beeper.Beeper()

In [3]:
#Также используем Progress Bar
from tqdm.notebook import tqdm

In [4]:
import ipython_exit

In [5]:
#Выставим опции pandas для удобства просмотра
pd.set_option('display.max_columns', 100)
pd.set_option('max_colwidth', 200)

In [6]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

Парсинг будем делать на локальной машине.

In [7]:
PATH_to_file = './data/'

Создадим словарь districs с наименованиями районов:

In [8]:
districts = ['almaty-medeuskij']
live_rooms = [1]

### Для парсинга сайта "krisha.kz" применим следующий подход:
- Применим фильтр по г. Алматы.  
- Применим фильтр по району и количеству комнат. В дальнейшем, возможно, по другим признакам.
- Попробуем прочесть доступное количество страниц.
- Дальше, как карта ляжет... )

Фильтр: Отдельно для каждого района и количества комнат:

In [9]:
district = 'almaty-medeuskij'
live_rooms = 1

In [14]:
#Верхний URL:
url = f'https://krisha.kz/prodazha/kvartiry/{district}/?das[live.rooms]={live_rooms}'
#Читаем страницу:
response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
if response.status_code != 200:
    raise BaseException("response code:" + str(response.status_code))
response.encoding = 'utf-8'
soup = BeautifulSoup(response.text, 'html.parser')

#Находим главную колонку
elem_mail_col = soup.find('div', class_='main-col')

#Находим количество объявлений по фильтру
elem_ads_count = elem_mail_col.find('div', class_='a-search-subtitle search-results-nb')
ads_count = int(elem_ads_count.text.split('Найдено ', 1)[1].split(' объявлен', 1)[0])
print('Количество объявлений = ', ads_count)

#Находим количество страниц
elem_paginator = elem_mail_col.find('nav', class_='paginator')
elem_paginator_btns = elem_paginator.find_all('a', class_ = 'paginator__btn')
pages_count1 = int(elem_paginator_btns[len(elem_paginator_btns) - 2].text.strip())

if ads_count % 20 == 0:
    pages_count2 = int(ads_count / 20)
else:
    pages_count2 = int(ads_count / 20) + 1
    
pages_count = max(pages_count1, pages_count2)
print(f'Количество страниц = ', pages_count)


#Главный цикл
flats_list_global = [] # здесь соберем список списков (эл-т - список параметров конкретноЙ квартиры)
pages_list = list(range(1, pages_count + 1)) # количество страниц для парсинга

with tqdm(total=len(pages_list)) as pbar:
    for p in pages_list:
        url = f'https://krisha.kz/prodazha/kvartiry/{district}/?das[live.rooms]={live_rooms}&page={p}'
        response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
        if response.status_code != 200:
            raise BaseException("response code" + str(response.status_code))
        response.encoding = 'utf-8'
        soup = BeautifulSoup(response.text, 'html.parser')
        
        #Находим главную колонку
        elem_mail_col = soup.find('div', class_='main-col')        
        
        elem_main_list = elem_mail_col.find('section', class_= 'a-list a-search-list a-list-with-favs')
        
        ads_id = []#Список id обычных объявлений. Этот список не нужен вообще-то
        
        #Ищем выделенную секцию
        elem_hilighted_section = elem_main_list.find('section', class_= 'highlighted-section')
        hs_ads_id = []#Список id объявлений из выделенной секции
        if elem_hilighted_section is not None:
            #Читаем объявления в выделенной секции
            elem_hilighted_section_ads = elem_hilighted_section.find_all('div', class_ = 'a-card')
            #Actually, the class is "a-card a-storage-live ddl_product ddl_product_link not-colored is-visible"
            print('hilighted_section_ads len = ', len(elem_hilighted_section_ads))
            for hs_ad in elem_hilighted_section_ads:#Для каждого объявления из выделенной секции
                hs_ads_id.append(hs_ad['data-id'])

            
        #Читаем обычные объявления
        elem_common_section_ads = elem_main_list.find_all('div', class_ = 'a-card')
        print('common_section_ads len = ', len(elem_common_section_ads))  
        for ad in elem_common_section_ads:#Для каждого объявления из выделенной секции
            if ad['data-id'] not in hs_ads_id:#Если этого объявления нет в выделенной секции (пропускаем спецобъявления)
                ads_id.append(ad['data-id'])#то добавляем объявление в список обычных объявлений.
                # Этот список не нужен вообще-то
                # А можно тут снять еще и дополнительные атрибуты, типа цвета объявления и количества просмотров и другие
                # и вызвать процедуру захода в это объявление
                #!! две процедуры: 1 пр. - обработка верхнего уровня объявления, 2 пр. (с параметрами) - заход в объявление
       
        print(ads_id)
        
        #Обновление ProgressBar и звуковой сигнал
        pbar.update(1)
        if p % 10 == 0:
            beeper.beep(frequency=100, secs=0.3, blocking=True)
        time.sleep(2)

Количество объявлений =  520
Количество страниц =  26


hilighted_section_ads len =  3
common_section_ads len =  20
['665347970', '664884298', '663310798', '664983312', '47507661', '665424687', '665424613', '665424652', '665414420', '665126691', '665423242', '662748502', '664801041', '665273222', '665214616', '664904788', '665196044']
common_section_ads len =  20
['665097238', '665173258', '665431681', '665373044', '665424719', '661854731', '665294410', '665120002', '664736548', '664953247', '665463354', '665462658', '665290079', '664013830', '665115704', '665115311', '665460529', '665459853', '664947780', '665458031']
common_section_ads len =  20
['664946141', '665457638', '665127150', '664926439', '665455861', '663375546', '664959877', '665453826', '665108712', '665260714', '665448196', '661276883', '665452972', '25290549', '665452140', '665451113', '660395176', '663629464', '664715795', '665436051']
common_section_ads len =  20
['665436051', '664809431', '664119542', '662344243', '665094296', '664885362', '665437917', '664512684', '66446

In [ ]:
ipython_exit.exit()

In [ ]:
brand = 'bmw'

In [ ]:
auto_list_global = [] # здесь соберем список списков (эл-т - список параметров конкретного автомобиля)
       
pages_list = list(range(1, 1201)) # количество страниц для парсинга 
with tqdm(total=len(pages_list)) as pbar:
    for i in pages_list: 
            
        url = f'https://auto.ru/moskva/cars/{brand}/used/?output_type=list&page={i}'
        response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
        if response.status_code != 200:
            raise BaseException("response code" + str(response.status_code))
        response.encoding = 'utf-8'
        soup = BeautifulSoup(response.text, 'html.parser')
        page = soup.find_all('div', class_='ListingItem-module__container')  

        for k in range(len(page)):     # проход по объявлениям на одной странице
            auto_html = page[k].find_all('meta')
            auto_list = []
            for j in range(len(auto_html)):   # обработка отдельного объявления на странице
                auto_list.append((str(auto_html[j])[15:]).split('"', 1)[0])

            auto_list_short = []
            auto_list_short.append(i) # Добавим пока номер страницы, чтобы узнать, откуда уже идут дубликаты

            # Убираем лишние параметры из списка    
            pos_out = {4,11,12,14,15,19}
            for n in range(len(auto_list)):   # удаление лишних параметров из объявления
                if n not in pos_out:
                    auto_list_short.append(auto_list[n].replace('\xa0', ' ')) 

            # Считываем пробег(расположен в другом месте)    
            all_km = page[k].find('div', class_='ListingItem-module__kmAge')
            km = str(all_km)[39:].split('<', 1)[0].replace('\xa0', ' ')
            auto_list_short.append(str(km))    # добавляем пробег
                
            auto_list_global.append(auto_list_short)    # добавляем список пар-ров автомобиля в список списков

        pbar.update(1)
        if i % 10 == 0:
            beeper.beep(frequency=340, secs=0.3, blocking=True)
        time.sleep(1.2)

In [ ]:
len(auto_list_global), len(auto_list_global[0])

In [ ]:
print(auto_list_global[:2])

Подберем в первом приближении имена столбцов в соответствии с именами в test. Как будем использовать информацию в столбце temp1, пока не понятно. 

In [ ]:
df = pd.DataFrame(auto_list_global,
                  columns=['page', 'bodyType', 'brand', 'color', 'fuelType', 'modelDate', 'model_name',
                           'numberOfDoors', 'productionDate', 'vehicleConfiguration', 'vehicleTransmission',
                           'price', 'temp1', 'engineDisplacement', 'enginePower', 'mileage'])

In [ ]:
df.head(3)

In [ ]:
df.info()

Удалим дубли:

In [ ]:
df.drop_duplicates(subset=['bodyType', 'brand', 'color', 'fuelType', 'modelDate', 'model_name', 'numberOfDoors',
                           'productionDate', 'vehicleConfiguration', 'vehicleTransmission', 'price', 'temp1',
                           'engineDisplacement', 'enginePower', 'mileage'], inplace=True)

In [ ]:
df.info()

Запишем данные в csv файл с соответствующим бренду именем:

In [ ]:
df.to_csv(PATH_to_file + brand + '.csv', index=False)

Повторив процедуру парсинга для каждой марки автомобиля из словаря, получим 12 файлов; из них потом получим один объединенный датасет train.

Считываем данные из отдельных csv и объединяем

In [ ]:
BMW = pd.read_csv(PATH_to_file + 'bmw.csv')
VOLKSWAGEN = pd.read_csv(PATH_to_file + 'volkswagen.csv')
NISSAN = pd.read_csv(PATH_to_file + 'nissan.csv')
MERCEDES = pd.read_csv(PATH_to_file + 'mercedes.csv')
TOYOTA = pd.read_csv(PATH_to_file + 'toyota.csv')
AUDI = pd.read_csv(PATH_to_file + 'audi.csv')
MITSUBISHI = pd.read_csv(PATH_to_file + 'mitsubishi.csv')
SKODA = pd.read_csv(PATH_to_file + 'skoda.csv')
VOLVO = pd.read_csv(PATH_to_file + 'volvo.csv')
HONDA = pd.read_csv(PATH_to_file + 'honda.csv')
INFINITI = pd.read_csv(PATH_to_file + 'infiniti.csv')
LEXUS = pd.read_csv(PATH_to_file + 'lexus.csv')

In [ ]:
frames = [BMW, VOLKSWAGEN, NISSAN, MERCEDES, TOYOTA, AUDI, MITSUBISHI, SKODA, VOLVO, HONDA, INFINITI, LEXUS]
train = pd.concat(frames)
train.sample(10)

In [ ]:
train.info()

Удалим пропуски, так как их мало и большое их количество находится в столбце с целевой переменной price

In [ ]:
train.dropna(inplace=True)

In [ ]:
train.info()

In [ ]:
train[train['price'].str.contains('RUB', na=False)]

In [ ]:
indices = train[train.price.str.lower().str.contains('rub', na=False)].index
train.drop(indices, inplace=True)

Преобразуем целевую переменную price в числовой формат

In [ ]:
train.price = train.price.astype('int64')

In [ ]:
train.info()

In [ ]:
train.sample(3)

Посмотрим на распределение количества объявлений по номерам страниц

In [ ]:
train[train.page < 200].page.hist()

Так как на сайте количество объявлений на странице почти фиксированное (37 - 38 объявлений) и мы удалили дубли, то можем заключить, что есть тренд к увеличению количества дублей с ростом номера страницы.

Запишем данные в итоговый csv файл

In [ ]:
train.to_csv(PATH_to_file + 'train.csv', index=False)

### В итоге мы получили тренировочный датасет train.csv с 26957 записей